## Решение задачи кредитного скоринга с использованием полностью гомоморфного шифрования

### Введение

В данной работе рассматривается ситуация, когда в банке возникла нехватка вычислительной мощности для решения задач машинного обучения, поэтому было принято решение арендовать для этих целей облачный сервер. 

Однако так как данные передаются на сторонний сервер, то возникает проблема безопасности данных, так как уязвимости в самом облаке, возможности несанкционированного доступа или даже неправильная настройка доступа к облаку могут привести к угрозам информационной безопасности, например, утечке данных, что снизит уровень доверия клиентов, а также приведёт к репутационным потерям и штрафам за нарушение правил информационной безопасности.

Решить данную проблему можно с помощью машинного обучения на зашифрованных данных, которое возможно с использованием полностью гомоморфного шифрования, позволяющего выполнять вычисления над зашифрованными данными без их расшифровки, что обеспечивает наивысший уровень безопасности, так как содержание данных никогда не раскрывается.

Демонстрация применения полностью гомоморфного шифрования для машинного обучения предлагается на примере решения задачи кредитного скоринга с предварительным шифрованием данных на стороне клиента (этап 1) и обучением и оценкой модели на зашифрованных данных на стороне сервера (этап 2). 

В качестве библиотеки полностью гомоморфного шифрования используется выбрана TenSEAL, основанная на Microsoft SEAL, которая в настоящее время является наиболее используемой библиотекой полностью гомоморфного шифрования. С помощью API она обеспечивает простоту использования языка Python, при этом сохраняя эффективность за счет реализации большинства операций с использованием C++.

### Этап 1 - Подготовка данных (клиент)

#### 1 Подготовка данных

##### Импорт библиотек

In [3]:
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tenseal as ts
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from time import time

##### Чтение данных
Набор данных [Credit score classification](https://www.kaggle.com/datasets/parisrohan/credit-score-classification/data), скачанный с Kaggle, выбран потому, что он включает в себя практически полную информацию о банковских клиентах, в том числе, возраст, профессию, годовой и ежемесячный доход, количество банковских счетов и так далее.

In [4]:
df = pd.read_csv('Credit score data.csv', low_memory=False)
df.tail()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.86622816574095,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.651610435322,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.8090832742814,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.1649785257098,Standard
99999,0x25fed,CUS_0x942c,August,Nicks,25,078-73-5990,Mechanic,39628.99_,3359.415833,4,...,Good,502.38,34.192463,31 Years and 10 Months,No,35.104023,167.1638651610451,!@9#%8,393.6736955618808,Poor


#### 2 Очистка данных

##### 2.1 Предварительная очистка данных

In [5]:
print('Размер набора данных: ', df.shape)

Размер набора данных:  (100000, 28)


In [6]:
df.describe(exclude=np.number).T

,count,unique,top,freq
ID,100000,100000,0x1602,1
Customer_ID,100000,12500,CUS_0xd40,8
Month,100000,8,January,12500
Name,90015,10139,Langep,44
Age,100000,1788,38,2833
SSN,100000,12501,#F%$D@*&8,5572
Occupation,100000,16,_______,7062
Annual_Income,100000,18940,36585.12,16
Num_of_Loan,100000,434,3,14386
Type_of_Loan,88592,6260,Not Specified,1408


Выводы:
* 12500 уникальных значений поля с идентификатором клиента Customer_ID означают то, что в наборе представлены данные 12500 клиентов.
* 8 уникальных значений поля с месяцем Month означают то, что данные представлены не за все месяцы.
* 1788 уникальных значений поля с возрастом Age выглядят странно.
* 12501 уникальных значений поля с номером социального страхования SNN при наличии данных о 12500 клиентах означают то, что у одного из клиентов представлен неправильный SNN.
* Данные содержат лишние символы, например, "_".

Удалим поля, не влияющие на кредитный рейтинг.

In [7]:
df.drop(['ID', 'Customer_ID', 'Name', 'SSN', 'Month'], axis=1, inplace=True)

Удалим лишние символы из данных.

In [8]:
def text_cleaning(data):
    if data is np.NaN or not isinstance(data, str):
        return data
    else:
        return str(data).strip('_ ,"')

In [9]:
df = df.map(text_cleaning)

Заменим не имеющие смысла значений на NaN.

In [10]:
df = df.replace(['', 'nan', '!@9#%8'], np.NaN)

Определим и преобразуем типы данных.

In [11]:
df.dtypes

Age                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                 object
dtype: object

Выводы:
* Данные представлены строковым и числовым типом.
* Числовые поля Age, Annual_Income, Num_of_Loan, Num_of_Delayed_Payment, Changed_Credit_Limit, Outstanding_Debt, Amount_invested_monthly, Monthly_Balance представлены неверным типом.

In [12]:
wrong_types = ['Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
               'Outstanding_Debt', 'Amount_invested_monthly', 'Monthly_Balance'
            ]

In [13]:
for col in wrong_types:
    df[col] = df[col].astype('float64')

С помощью функции convert_to_month() конвертируем значения поля Credit_History_Age, представленные годами и месяцами, в месяцы (например, "2 Years and 7 Months" в "19") и преобразуем в тип float64.

In [14]:
def convert_to_months(history):
    if pd.notnull(history):
        years = int(history.split(' ')[0])
        months = int(history.split(' ')[3])
        return (years * 12) + months
    return 0

In [15]:
df['Credit_History_Age'] = df['Credit_History_Age'].apply(lambda x: convert_to_months(x)).astype(float)

##### 2.2 Обработка количественных данных

In [16]:
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns.to_list()

for col in numerical_columns:
    print(df[col].value_counts())
    print('_______________________')

Age
38.0      2994
28.0      2968
31.0      2955
26.0      2945
32.0      2884
          ... 
1908.0       1
4583.0       1
7549.0       1
3119.0       1
1342.0       1
Name: count, Length: 1728, dtype: int64
_______________________
Annual_Income
17816.75       16
22434.16       16
40341.16       16
17273.83       16
109945.32      16
               ..
17079092.00     1
1910572.00      1
20179076.00     1
7980216.00      1
8299495.00      1
Name: count, Length: 13487, dtype: int64
_______________________
Monthly_Inhand_Salary
6769.130000    15
6358.956667    15
2295.058333    15
6082.187500    15
3080.555000    14
               ..
1087.546445     1
3189.212103     1
5640.117744     1
7727.560450     1
2443.654131     1
Name: count, Length: 13235, dtype: int64
_______________________
Num_Bank_Accounts
6       13001
7       12823
8       12765
4       12186
5       12118
        ...  
1626        1
1470        1
887         1
211         1
697         1
Name: count, Length: 943, dtype: 

Обработаем отрицательные значения, заменив их на NaN.

In [17]:
for col in numerical_columns:
    df[col] = df[col].map(lambda x: np.NaN if x < 0 else x)

В поле Age представлено 1788 уникальных значений, но возраст должны быть целым числом от 18 до 85 (максимальный возраст заёмщика).

In [18]:
df = df.loc[(df['Age'] >= 18 ) & (df['Age'] <= 85)]

Также отбросим выбросы и ограничим диапазон других значений, так как, например, у клиента не может быть 1118 кредитных карт или 2397 кредитных запросов.

In [19]:
df = df[df['Interest_Rate'] <= 50]
df = df[df['Num_of_Loan'] <= 12]
df = df[df['Num_Credit_Card'] <= 10]
df = df[df['Num_Bank_Accounts'] <= 10]
df = df[df['Delay_from_due_date'] <= 60]
df = df[df['Changed_Credit_Limit'] <= 30]
df = df[df['Num_Credit_Inquiries'] <= 12]
df = df[df['Total_EMI_per_month'] <= 200]
df = df[df['Outstanding_Debt'] <= 1500]

Рассчитаем пропущенные значения в полях Monthly_Inhand_Salary, Num_of_Delayed_Payment, Amount_invested_monthly и Monthly_Balance как среднее арифметическое.

In [20]:
null_num_columns = ['Monthly_Inhand_Salary', 'Num_of_Delayed_Payment', 'Amount_invested_monthly', 'Monthly_Balance']
imputer = SimpleImputer(strategy='mean')
df[null_num_columns] = imputer.fit_transform(df[null_num_columns])

##### 2.3 Обработка категориальных данных

In [21]:
categorical_columns = df.select_dtypes(include=['object']).columns.to_list()

for col in categorical_columns:
    print(df[col].value_counts())
    print('_______________________')

Occupation
Lawyer           3070
Architect        2950
Media_Manager    2774
Teacher          2742
Accountant       2740
Developer        2721
Entrepreneur     2684
Journalist       2683
Musician         2683
Mechanic         2681
Scientist        2679
Manager          2650
Engineer         2638
Doctor           2637
Writer           2578
Name: count, dtype: int64
_______________________
Type_of_Loan
Not Specified                                                                                                                                   1026
Personal Loan                                                                                                                                    972
Student Loan                                                                                                                                     957
Credit-Builder Loan                                                                                                                              933


Поле Payment_of_Min_Amount может содержать только значения "Yes" или "No", но не может содержать "NM".

In [22]:
df.loc[df['Payment_of_Min_Amount'] == 'NM', 'Payment_of_Min_Amount'] = np.NaN

Значения поля Type_of_Loan содержат ненужный союз "and".

In [23]:
df['Type_of_Loan'] = df['Type_of_Loan'].apply(lambda x: x.lower().replace('and ', '').replace(', ', ',').strip() if pd.notna(x) else x)

В поле Credit_Score нет пропущенных значений и оно принимает значения Good, Poor и Standard, поэтому сразу преобразуем их в числа.

In [24]:
categories = ['Poor', 'Standard', 'Good']
encoder = OrdinalEncoder(categories=[categories])
df['Credit_Score'] = encoder.fit_transform(df[['Credit_Score']])
df['Credit_Score'].value_counts()

Credit_Score
1.0    27179
2.0    10786
0.0     6073
Name: count, dtype: int64

Закодируем числами строковые категориальные данные.

In [25]:
cat_columns = ['Payment_of_Min_Amount','Payment_Behaviour', 'Type_of_Loan', 'Occupation','Credit_Mix']
encoder = ce.LeaveOneOutEncoder()
for col in cat_columns:
    df[col] = encoder.fit_transform(df[col], df['Credit_Score'])

##### 2.5 Проверка на наличие пропущенных и повторяющихся значений

In [26]:
if df.isnull().sum().sum() == 0:
    print('Пропущенных значений нет.')
else:
    print('Пропущенных значений: ', df.isnull().sum().sum())

Пропущенных значений нет.


In [27]:
if len(df)- len(df.drop_duplicates()) == 0:
    print('Повторяющихся значений нет.')
else:
    print('Повторяющихся значений: ', len(df)- len(df.drop_duplicates()))

Повторяющихся значений нет.


##### 2.6 Нормализация данных

In [28]:
columns_list = df.columns.to_list()
scaler = MinMaxScaler()

for col in columns_list:
    df[col] = scaler.fit_transform(df[[col]])

df.describe()

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
count,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,...,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000,44038.000000
mean,0.466982,0.525277,0.007040,0.269427,0.438860,0.478739,0.278289,0.243853,0.553697,0.247646,...,0.815815,0.516099,0.406787,0.608087,0.613556,0.293204,0.062906,0.533045,0.276453,0.553511
std,0.280047,0.244176,0.058854,0.194615,0.230519,0.167751,0.178117,0.187159,0.150415,0.154357,...,0.171348,0.295168,0.178992,0.269255,0.457062,0.267136,0.197074,0.353039,0.140076,0.304706
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.236842,0.408005,0.000627,0.122199,0.300000,0.400000,0.151515,0.111111,0.500000,0.133333,...,0.689881,0.258500,0.259180,0.487624,0.000211,0.077855,0.008070,0.364681,0.184810,0.500000
50%,0.447368,0.514616,0.001245,0.230258,0.400000,0.500000,0.242424,0.222222,0.581359,0.216667,...,0.817089,0.518681,0.406904,0.638614,0.999733,0.228203,0.014815,0.579996,0.232323,0.500000
75%,0.684211,0.696283,0.002814,0.353972,0.600000,0.600000,0.393939,0.333333,0.589623,0.350000,...,0.999886,0.784529,0.553138,0.819307,0.999866,0.442293,0.030630,0.841845,0.326868,0.500000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


##### 2.7 Сохранение обработанных данных в формате CSV

In [29]:
sample_df = df.sample(frac=0.1)
sample_df.to_csv("Prepared credit score data.csv", index=False)

#### 3 Шифрование данных

##### 3.1 Разделение данных

In [30]:
x = sample_df.drop(columns=['Credit_Score'])
y = sample_df['Credit_Score'].copy()

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

##### 3.2 Шифрование данных

Для шифрования данных предлагается использовать схему CKKS, так как она позволяет работать с вещественными числами, которыми представлены данные. 
Параметры шифрования схемы CKKS включают в себя:
* степень полиномиального модуля;
* размеры модуля коэффициента.

Определим значения параметров шифрования, для чего сначала определим количество необходимых операций умножения:
* для операции скалярного произведения - 1;
* для аппроксимации сигмоидной функции - 2;
* для осуществления обратного распространения ошибки - 3.  

Таким образом, мультипликативная глубина равна 1 + 2 + 3 = 6, что определяет количество чисел, состовляющих модуль коэффцииента. 

Степень полиномиального модуля poly_mod_grade, которую ещё называют коэффициентом масштабирования, должна быть степенью двойки (1024, 2048, 4096, 8192 и т. д.), так как она определяет точность кодирования для двоичного представления числа, то есть контролирует точность дробной части, поскольку это значение, на которое умножаются открытые тексты перед кодированием в полином целочисленных коэффициентов.  
Приняв желаемый уровнь защищенности, эквивалентный AES, равный 128 битам, степень полиномиального модуля должна быть не ниже 8192, что позволит группировать до 4096 значений в одном зашифрованном тексте.

Размеры модуля коэффициента coeff_mod_bit_sizes представляют собой список двоичных размеров, используя который TenSEAL генерирует список простых чисел этих двоичных размеров. Так как желаемый уровень защищённости составляет 128 бит, количество умножений равно 6, а 128 / 6 = 21.3, то в качестве двоичного размера следует использовать 21 бит. При этом двоичные размеры в количестве, равном количеству умножений, определяются в coeff_mod_bit_sizes[1:-1] и должны быть равными друг другу, а первый и последний размер в данном списке должны быть больше их по размеру.  
Таким образом, размеры модуля коэффициента [40, 21, 21, 21, 21, 21, 21, 40] обозначают то, что модуль коэффициента будет содержать 8 простых чисел: первое и последнее по 40 бит и остальные по 21 бит. 

In [32]:
poly_mod_grade = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]

Создадим TenSEALContext для указания схемы и параметров. По умолчанию создаются ключи релинеаризации, а также влючена автоматическая релинеаризация и масштабирование.

In [33]:
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_grade, -1, coeff_mod_bit_sizes)

Также укажем параметр global_scale, который используется в качестве значения масштаба по умолчанию, и который равен 21 биту.

In [34]:
ctx_training.global_scale = 2 ** 21

Для выполнения операции скалярного произведения также требуется определить ключи Галуа.

In [35]:
ctx_training.generate_galois_keys()

Создадим тензоры.

In [36]:
x_train = torch.tensor(x_train.values).to(torch.float32)
x_test = torch.tensor(x_test.values).to(torch.float32)
y_train = torch.tensor(y_train.values).to(torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values).to(torch.float32).unsqueeze(1)

Зашифруем тензоры c помощью функции encrypt_data(), которая зашифровывает принимаемый тензор и сериализует получившийся вектор, который после этого записывается в файл и архивируется.

In [37]:
import os
from zipfile import ZipFile

def encrypt_data(tensor, filename, ctx_training):
    t_start = time()
    rows = x_train.shape[0]
    zip_name = f'encrypted data//{filename}.zip'
    
    with ZipFile(zip_name, 'w') as filezip:   
        for i in range(rows):
            vector = ts.ckks_vector(ctx_training, tensor[i].tolist()).serialize()
            filename_with_num = f'encrypted data//{filename}_{i}.hex'
            with open(filename_with_num, 'wb') as file:
                file.write(vector)
            filezip.write(filename_with_num)
            os.remove(filename_with_num)
    t_end = time()
    print(f'Шифрование заняло {int(t_end - t_start)} секунд')

In [38]:
encrypt_data(x_train, 'x_train', ctx_training)

Шифрование заняло 61 секунд


In [39]:
encrypt_data(y_train, 'y_train', ctx_training)

Шифрование заняло 58 секунд
